In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/Users/shraawanilattoo/Library/CloudStorage/OneDrive-NorthCarolinaStateUniversity/NCSU/Fellowship/tennis_atp-master/atp_matches_2020_working.csv')
df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2020-8888,Atp Cup,Hard,24,A,20200106,300,104925,NaN,NaN,...,51.0,39.0,6.0,10.0,6.0,8.0,2.0,9055.0,1.0,9985.0
1,2020-8888,Atp Cup,Hard,24,A,20200106,299,105138,NaN,NaN,...,35.0,21.0,6.0,9.0,5.0,10.0,10.0,2335.0,34.0,1251.0
2,2020-8888,Atp Cup,Hard,24,A,20200106,298,104925,NaN,NaN,...,57.0,35.0,25.0,14.0,6.0,11.0,2.0,9055.0,5.0,5705.0
3,2020-8888,Atp Cup,Hard,24,A,20200106,297,105583,NaN,NaN,...,54.0,39.0,14.0,12.0,0.0,1.0,34.0,1251.0,17.0,1840.0
4,2020-8888,Atp Cup,Hard,24,A,20200106,296,104745,NaN,NaN,...,55.0,37.0,10.0,14.0,1.0,5.0,1.0,9985.0,18.0,1775.0


In [3]:
def tourney_name(name):
    short_df = df[df['tourney_name'].str.contains(name, case=False, na=False)]
    return short_df

short_df=tourney_name(name=input("Name of the tournament: "))
short_df.head(10)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
693,2020-560,Us Open,Hard,128,G,20200831,100,104925,1.0,NaN,...,53.0,29.0,15.0,12.0,12.0,19.0,1.0,10860.0,109.0,524.0
694,2020-560,Us Open,Hard,128,G,20200831,101,106378,NaN,NaN,...,72.0,56.0,18.0,19.0,10.0,16.0,44.0,1050.0,54.0,965.0
695,2020-560,Us Open,Hard,128,G,20200831,102,111581,NaN,NaN,...,56.0,32.0,25.0,17.0,4.0,10.0,186.0,281.0,68.0,776.0
696,2020-560,Us Open,Hard,128,G,20200831,103,105526,28.0,NaN,...,66.0,37.0,14.0,14.0,10.0,18.0,29.0,1405.0,106.0,546.0
697,2020-560,Us Open,Hard,128,G,20200831,104,105807,20.0,NaN,...,101.0,79.0,15.0,22.0,12.0,16.0,27.0,1500.0,86.0,685.0
698,2020-560,Us Open,Hard,128,G,20200831,105,106283,NaN,NaN,...,71.0,52.0,16.0,19.0,5.0,11.0,198.0,264.0,110.0,522.0
699,2020-560,Us Open,Hard,128,G,20200831,106,105575,NaN,NaN,...,60.0,44.0,38.0,22.0,8.0,13.0,65.0,809.0,132.0,423.0
700,2020-560,Us Open,Hard,128,G,20200831,107,105449,NaN,NaN,...,134.0,99.0,23.0,28.0,9.0,11.0,64.0,825.0,22.0,1805.0
701,2020-560,Us Open,Hard,128,G,20200831,108,133430,12.0,NaN,...,100.0,58.0,23.0,18.0,10.0,14.0,17.0,2075.0,205.0,252.0
702,2020-560,Us Open,Hard,128,G,20200831,109,126952,NaN,NaN,...,70.0,45.0,25.0,20.0,2.0,8.0,73.0,742.0,187.0,281.0


In [ ]:
# Use only first round matches: 'R128', 'R64', 'R32' (whichever appears as first round)
first_rounds = ['R128', 'R64', 'R32']  # These are typical starting rounds depending on tournament size
first_round_df = short_df[short_df['round'].isin(first_rounds)].copy()

# Initialize Elo ratings
elo_ratings = {}
K = 32

def get_elo(player):
    return elo_ratings.get(player, 1500)

def expected_score(player_elo, opponent_elo):
    return 1 / (1 + 10 ** ((opponent_elo - player_elo) / 400))

def update_elo(winner, loser):
    winner_elo = get_elo(winner)
    loser_elo = get_elo(loser)
    expected_win = expected_score(winner_elo, loser_elo)
    expected_loss = expected_score(loser_elo, winner_elo)

    winner_elo += K * (1 - expected_win)
    loser_elo += K * (0 - expected_loss)

    elo_ratings[winner] = winner_elo
    elo_ratings[loser] = loser_elo

# Apply Elo updates only on first round matches
for _, row in first_round_df.iterrows():
    winner = row['winner_name']
    loser = row['loser_name']
    update_elo(winner, loser)

# Create Elo reseeding DataFrame
elo_df = pd.DataFrame(elo_ratings.items(), columns=['winner_name', 'elo']).sort_values(by='elo', ascending=False).reset_index(drop=True)
elo_df['reseeding'] = elo_df.index + 1


# Merge tournament_df with elo_df on 'player name' to get the reseeding info
merged_df = short_df.merge(elo_df[['winner_name', 'reseeding']], on='winner_name', how='left')

# Replace 'winner seed' with 'reseeding'
merged_df['winner_seed'] = merged_df['reseeding']

# Optionally drop the 'reseeding' column if not needed
merged_df = merged_df.drop(columns=['reseeding'])

# Save the updated DataFrame to a new CSV
merged_df.to_csv('updated_tournament.csv', index=False)
